In [1]:
from data_gen import *
from models import *

In [2]:
# data generator
dg = DataGen(data="data/wol-aligned.txt")

# length of a word
n_input_length = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len


6


In [3]:
print("Total train data: ", len(dg.words) * .7)
batch_size = 128
# number of batches to train
n_batches = int(len(dg.words) * .7 / batch_size) 

# python generator to generate training data at each request
# E.x word_matrix, feature = next(gen)
gen = dg.rnn_gen_data(batch_size=batch_size, n_batches=n_batches)

Total train data:  564292.3999999999


In [6]:
# infenc - inference encoder model
# infdec - inference decoder model
# train - training model that combines both
# n_input_length - the length of the input and the output
# word_feat_len - the length of the word feature vector
# n_units - size of the hidden memory in the RNN
train, infenc, infdec = rnn_multi_input(n_input_length, n_input_length, dg.word_feat_len + 3, 256, 256)
train.compile(optimizer='adam', loss='categorical_crossentropy')


In [7]:
# model train given the data generator, how many batches and number of epochs
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 4)

W0717 08:24:28.047051 140528905537344 deprecation.py:323] From /home/leo/anaconda3/envs/tf2/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/4
4408/4408 [==============================] - 234s 53ms/step - loss: 0.1587
Epoch 2/4
4408/4408 [==============================] - 232s 53ms/step - loss: 0.0041
Epoch 3/4
 524/4408 [==>...........................] - ETA: 3:24 - loss: 0.0097

KeyboardInterrupt: 

In [26]:
# test_n_batches, test_batch_size =  int(len(dg.words) * .7 / batch_size), batch_size  
test_n_batches, test_batch_size = 30, 10 

# data generator for test data
test_gen = dg.rnn_gen_data(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

In [27]:
def predict(infenc, infdec, inputs, n_steps, cardinality):
    # encode
    h, c = infenc.predict(inputs)
    # start of sequence input
    start = [0.0 for _ in range(cardinality)]
#     start[0] = 1
    target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
#     print(len(state))
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq, h, c])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [31]:
# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    # get data from test data generator
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        word_features = X3[j].reshape((1, X3.shape[1])) 
        root_word_matrix = X1[j].reshape((1, X1.shape[1], X1.shape[2]))
#         word_index = X4[j].reshape((1, X4.shape[1]))
        # predicts the target word given root word and features
        target = predict(infenc, infdec, [root_word_matrix, word_features], n_steps_out, n_input_length)
        root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        
        # checks if the predicted and the real words are equal
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target):
            correct += 1
        else:
            print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
#         if root.strip() in targetS.strip():
#             in_word += 1
#     print(b, root, word, targetS)
    total += test_batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

caal            caalissageeta 		 caalissigag
shaaramux       shaaramuxidori 		 shaaramuxori
loggom          loggomogee 		 loggomodogee
doom            doomettaas 		 doomettida
morkkett        morkkettari 		 morkkettidari
gomet           gometissidari 		 gometissari
booccot         booccotidogaa 		 booccotogaa
toylatt         toylattssidogee 		 toylattssodogee
cecc            ceccissidogee 		 ceccissodogee
tamaar          tamaarissaree 		 tamaarissidaree
qunxx           qunxxssaasi 		 qunxxssaas
Exact Accuracy: 96.33%


In [ ]:
# lines = open("data/wol-multi.txt").readlines()
for i in range(10):
    print(dg.roots[i], dg.words[i], dg.word_indexes[i])

In [ ]:
# word2feat, word2root= {}, {}

In [ ]:
# for line in lines:
#     line = line[:-1].split(' ')
#     word2feat[line[0]] = line[1]
#     word2root[line[0]] = line[2]

In [ ]:
# check = {}
# for word in word2feat:
#     root = word2feat[word]
#     feat = word2root[word]
#     key = root + " " + feat
#     if key not in check:
#         check[key] = []
#     check[key].append(word)
    

In [ ]:
# counter = 0
# file = open("data/wol-m.txt", "w")
# for key in check.keys():
#     words = check[key]
#     for i in range(len(words)):
#         word = words[i]
#         line = "{0} {1} {2}\n".format(word, key, i)
#         file.write(line)
# file.close()